In [ ]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
sns.set_style("whitegrid")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


file_name = 'tmdb_movie_details.csv'
df = pd.read_csv(file_name, on_bad_lines='skip')

print(f"--- Successfully loaded '{file_name}' ---")
print(f"Total rows: {len(df)}, Total columns: {len(df.columns)}")
print("\n")

--- Successfully loaded 'tmdb_movie_details.csv' ---
Total rows: 191614, Total columns: 18




In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191614 entries, 0 to 191613
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   imdb_id                191614 non-null  object 
 1   tmdb_id                191614 non-null  int64  
 2   title                  191614 non-null  object 
 3   release_date           188802 non-null  object 
 4   vote_average           191614 non-null  float64
 5   runtime                191614 non-null  int64  
 6   genres                 172471 non-null  object 
 7   director               185179 non-null  object 
 8   top_cast               182443 non-null  object 
 9   actors_other_works     191614 non-null  object 
 10  directors_other_works  191614 non-null  object 
 11  overview               182112 non-null  object 
 12  original_language      191614 non-null  object 
 13  production_companies   133580 non-null  object 
 14  production_countries   160637 non-nu

# Remove unhelpful and unusable columns due to problem's constraint

In [ ]:
columns_to_drop = ['vote_average','overview']
df = df.drop(columns=columns_to_drop)

print("--- Columns dropped successfully ---")
df.info()

--- Columns dropped successfully ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191614 entries, 0 to 191613
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   imdb_id                191614 non-null  object
 1   tmdb_id                191614 non-null  int64 
 2   title                  191614 non-null  object
 3   release_date           188802 non-null  object
 4   runtime                191614 non-null  int64 
 5   genres                 172471 non-null  object
 6   director               185179 non-null  object
 7   top_cast               182443 non-null  object
 8   actors_other_works     191614 non-null  object
 9   directors_other_works  191614 non-null  object
 10  original_language      191614 non-null  object
 11  production_companies   133580 non-null  object
 12  production_countries   160637 non-null  object
 13  budget                 191614 non-null  int64 
 14  revenue        

# Convert actor and director credits score into separate columns and add an average actor score, top actor score, number of actor columns

In [ ]:
def extract_actor_scores(json_str):
    """Parses the JSON string and extracts top 5 actor 'prior_counts'."""
    try:
        data = json.loads(json_str)
        # Extract the 'prior_count' for each actor in the list
        scores = [item.get('prior_count', 0) for item in data]

        # Ensure we have exactly 5 scores:
        # If > 5, take first 5. If < 5, pad with 0s.
        scores = scores[:5] + [0] * (5 - len(scores))
        return scores
    except (json.JSONDecodeError, TypeError, AttributeError):
        # Return zeros if data is missing or corrupt
        return [0, 0, 0, 0, 0]

def extract_director_score(json_str):
    """Parses JSON string and extracts the 1st director's 'prior_count'."""
    try:
        data = json.loads(json_str)
        if data and isinstance(data, list):
            return data[0].get('prior_count', 0)
        return 0
    except (json.JSONDecodeError, TypeError, AttributeError):
        return 0
actor_scores = df['actors_other_works'].apply(extract_actor_scores)
df[['actor_1_score', 'actor_2_score', 'actor_3_score', 'actor_4_score', 'actor_5_score']] = pd.DataFrame(actor_scores.tolist(), index=df.index)
df['director_score'] = df['directors_other_works'].apply(extract_director_score)
df = df.drop(columns=['actors_other_works','directors_other_works'])
df['average_actor_score'] = df[['actor_1_score', 'actor_2_score', 'actor_3_score', 'actor_4_score', 'actor_5_score']].mean(axis=1)
df['top_actor_score'] = df[['actor_1_score', 'actor_2_score', 'actor_3_score', 'actor_4_score', 'actor_5_score']].max(axis=1)
def count_actors(cast_str):
    if pd.isna(cast_str) or cast_str == 'Missing value' or cast_str == '':
        return 0

    # Split by comma and count individual elements
    names = [name.strip() for name in str(cast_str).split(',') if name.strip()]
    return len(names)

# Fill the new column
df['number_of_actors'] = df['top_cast'].apply(count_actors)
df.head()

,imdb_id,tmdb_id,title,release_date,runtime,genres,director,top_cast,original_language,production_companies,production_countries,budget,revenue,status,actor_1_score,actor_2_score,actor_3_score,actor_4_score,actor_5_score,director_score,average_actor_score,top_actor_score,number_of_actors
0,tt0015724,286375,Dama de Noche,1993-05-18,96,"Drama, Mystery, Romance, Thriller",Eva López Sánchez,"Rafael Sánchez Navarro, Cecilia Toussaint, Mig...",es,Centro de Capacitación Cinematográfica (CCC),Mexico,0,0,Released,13,7,12,16,70,2,23.6,70,5
1,tt0035423,11232,Kate & Leopold,2001-12-25,118,"Romance, Comedy, Fantasy",James Mangold,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",en,"Konrad Pictures, Miramax",United States of America,48000000,76019048,Released,51,43,58,31,31,13,42.8,58,5
2,tt0036606,73069,"Another Time, Another Place",1983-05-13,96,"Drama, Romance, War",Michael Radford,"Phyllis Logan, Giovanni Mauriello, Denise Coff...",en,"Umbrella, Associated-Rediffusion Television, T...",United Kingdom,0,0,Released,4,0,20,22,1,3,9.4,22,5
3,tt0039442,133498,"Speak, Silent One",1973-06-01,88,Drama,Manuel Gutiérrez Aragón,"José Luis López Vázquez, Kiti Mánver, Hanna Ha...",es,"Filmverlag der Autoren, Elías Querejeta PC","Spain, Germany",0,0,Released,144,2,0,10,2,3,31.6,144,5
4,tt0044952,235186,The Citizen,1977-09-20,112,Drama,Ritwik Ghatak,"Satindra Bhattacharya, Prabhadevi, Sova Sen, K...",bn,NaN,India,0,0,Released,10,4,26,12,1,48,10.6,26,5


# Convert to correct datatype, missing number value is converted to 0 for numeric columns

In [ ]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

float_cols = ['average_actor_score']

for col in float_cols:
    # 1. Convert to numeric. Errors (like bad text) become NaN.
    # 2. Fill all NaN values with 0.
    # 3. Set the type to float.
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
    print(f"Converted '{col}' to float and filled missing with 0.")


int_cols = ['budget', 'revenue', 'runtime', 'actor_1_score','actor_2_score','actor_3_score','actor_4_score','actor_5_score','director_score']

for col in int_cols:
    # 1. Convert to numeric. Errors (like bad text) become NaN.
    # 2. Fill all NaN values with 0.
    # 3. Set the type to int64 (a whole number).
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(np.int64)
    print(f"Converted '{col}' to int64 and filled missing with 0.")

print("\nAll numeric conversions complete.\n")
df.info()

Converted 'average_actor_score' to float and filled missing with 0.
Converted 'budget' to int64 and filled missing with 0.
Converted 'revenue' to int64 and filled missing with 0.
Converted 'runtime' to int64 and filled missing with 0.
Converted 'actor_1_score' to int64 and filled missing with 0.
Converted 'actor_2_score' to int64 and filled missing with 0.
Converted 'actor_3_score' to int64 and filled missing with 0.
Converted 'actor_4_score' to int64 and filled missing with 0.
Converted 'actor_5_score' to int64 and filled missing with 0.
Converted 'director_score' to int64 and filled missing with 0.

All numeric conversions complete.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191614 entries, 0 to 191613
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   imdb_id               191614 non-null  object        
 1   tmdb_id               191614 non-null  int64         
 2   titl

# Filling in missing value for text columns

In [ ]:
text_date_cols_to_fill = [
    'release_date',
    'genres',
    'director',
    'top_cast',
    'production_companies',
    'production_countries',
    'original_language',
    'original_title'
    'status'
]

for col in text_date_cols_to_fill:
    if col in df.columns:
        df[col] = df[col].fillna('Missing value')
        print(f"Filled missing values in '{col}' with 'Missing value'.")
    else:
        print(f"Column '{col}' not found in DataFrame.")

print("\nAll specified text and date column missing values filled.")
df.info()

Filled missing values in 'release_date' with 'Missing value'.
Filled missing values in 'genres' with 'Missing value'.
Filled missing values in 'director' with 'Missing value'.
Filled missing values in 'top_cast' with 'Missing value'.
Filled missing values in 'production_companies' with 'Missing value'.
Filled missing values in 'production_countries' with 'Missing value'.
Filled missing values in 'original_language' with 'Missing value'.
Column 'original_titlestatus' not found in DataFrame.

All specified text and date column missing values filled.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191614 entries, 0 to 191613
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   imdb_id               191614 non-null  object 
 1   tmdb_id               191614 non-null  int64  
 2   title                 191614 non-null  object 
 3   release_date          191614 non-null  object 
 4   runtime           

# handling multivalue field in 'production_countries' by only keeping the first value

In [ ]:
#Remove all only keep the first production country
def get_first_country(countries_string):
        if not countries_string:
            return 'Missing value'

        try:
            countries_list = countries_string.split(',')
            first_country = countries_list[0]

            first_country = first_country.strip()

            if not first_country:
                 if len(countries_list) > 1:
                    first_country = countries_list[1].strip()
                 else:
                    return 'Missing value'

            return first_country

        except Exception as e:
            return 'Missing value'
df['main_production_country'] = df['production_countries'].apply(get_first_country)
df = df.drop(columns=['production_countries'])
df.head()

,imdb_id,tmdb_id,title,release_date,runtime,genres,director,top_cast,original_language,production_companies,budget,revenue,status,actor_1_score,actor_2_score,actor_3_score,actor_4_score,actor_5_score,director_score,average_actor_score,top_actor_score,number_of_actors,main_production_country
0,tt0015724,286375,Dama de Noche,1993-05-18 00:00:00,96,"Drama, Mystery, Romance, Thriller",Eva López Sánchez,"Rafael Sánchez Navarro, Cecilia Toussaint, Mig...",es,Centro de Capacitación Cinematográfica (CCC),0,0,Released,13,7,12,16,70,2,23.6,70,5,Mexico
1,tt0035423,11232,Kate & Leopold,2001-12-25 00:00:00,118,"Romance, Comedy, Fantasy",James Mangold,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",en,"Konrad Pictures, Miramax",48000000,76019048,Released,51,43,58,31,31,13,42.8,58,5,United States of America
2,tt0036606,73069,"Another Time, Another Place",1983-05-13 00:00:00,96,"Drama, Romance, War",Michael Radford,"Phyllis Logan, Giovanni Mauriello, Denise Coff...",en,"Umbrella, Associated-Rediffusion Television, T...",0,0,Released,4,0,20,22,1,3,9.4,22,5,United Kingdom
3,tt0039442,133498,"Speak, Silent One",1973-06-01 00:00:00,88,Drama,Manuel Gutiérrez Aragón,"José Luis López Vázquez, Kiti Mánver, Hanna Ha...",es,"Filmverlag der Autoren, Elías Querejeta PC",0,0,Released,144,2,0,10,2,3,31.6,144,5,Spain
4,tt0044952,235186,The Citizen,1977-09-20 00:00:00,112,Drama,Ritwik Ghatak,"Satindra Bhattacharya, Prabhadevi, Sova Sen, K...",bn,Missing value,0,0,Released,10,4,26,12,1,48,10.6,26,5,India


# **Extracting id list to crawl box office mojo**

In [ ]:
norevenue = df[df['revenue'] == 0]
hasrevenue = df[df['revenue'] != 0]
print(norevenue.shape)
print(hasrevenue.shape)
norevenue.info()

(173258, 23)
(18356, 23)
<class 'pandas.core.frame.DataFrame'>
Index: 173258 entries, 0 to 191613
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   imdb_id                  173258 non-null  object 
 1   tmdb_id                  173258 non-null  int64  
 2   title                    173258 non-null  object 
 3   release_date             173258 non-null  object 
 4   runtime                  173258 non-null  int64  
 5   genres                   173258 non-null  object 
 6   director                 173258 non-null  object 
 7   top_cast                 173258 non-null  object 
 8   original_language        173258 non-null  object 
 9   production_companies     173258 non-null  object 
 10  budget                   173258 non-null  int64  
 11  revenue                  173258 non-null  int64  
 12  status                   173258 non-null  object 
 13  actor_1_score            173258 non-nul

## Filtering for movies that likely has budget availble to reduce crawl time, with criteria:


*   has budget

OR


*   no missing text columns excluding production company
*   date < 2025-10-01
*   production country isnt USA





In [ ]:
filtered_df = norevenue.query("budget!=0 or release_date != 'Missing value' and genres!= 'Missing value' and director != 'Missing value' and top_cast != 'Missing value'  and main_production_country != 'United States of America' and main_production_country!='Missing value' and release_date !='Missing value'").copy()
filtered_df['release_date'] = pd.to_datetime(filtered_df['release_date'],errors = 'coerce')
filtered_df = filtered_df[filtered_df['release_date'] < '2025-10-01']
print(filtered_df.shape)

(111333, 23)


In [ ]:
id_list = filtered_df[['imdb_id','tmdb_id','title','main_production_country']]
id_list.head()


,imdb_id,tmdb_id,title,main_production_country
0,tt0015724,286375,Dama de Noche,Mexico
2,tt0036606,73069,"Another Time, Another Place",United Kingdom
3,tt0039442,133498,"Speak, Silent One",Spain
4,tt0044952,235186,The Citizen,India
5,tt0048550,257317,Rendez-vous of the Docks,France


## Extracting result

In [ ]:
id_list.to_csv('missing_list.csv', index=False)

# Cleaning box office mojo and merging it into current dataset

In [ ]:
df_extra = pd.read_csv('boxoffice_results.csv')
df_extra.head()

,imdb_id,tmdb_id,title,main_production_country,budget,worldwide_revenue
0,tt0015724,286375,Dama de Noche,Mexico,NaN,NaN
1,tt0036606,73069,"Another Time, Another Place",United Kingdom,NaN,NaN
2,tt0039442,133498,"Speak, Silent One",Spain,NaN,NaN
3,tt0044952,235186,The Citizen,India,NaN,NaN
4,tt0048550,257317,Rendez-vous of the Docks,France,NaN,NaN


In [ ]:
def clean_currency(x):
    """
    Removes '$', ',', and whitespace from a string and converts to float.
    Returns the original value if it's not a string (e.g., already a number or NaN).
    """
    if isinstance(x, str):
        cleaned = x.replace('$', '').replace(',', '').strip()
        try:
            return float(cleaned)
        except ValueError:
            return None
    return x


df_extra['budget'] = df_extra['budget'].apply(clean_currency)
df_extra['worldwide_revenue'] = df_extra['worldwide_revenue'].apply(clean_currency)

In [ ]:
df = df.merge(df_extra[['imdb_id', 'tmdb_id', 'budget', 'worldwide_revenue']], on=['imdb_id', 'tmdb_id'], how='left', suffixes=('', '_extra'))


df['revenue'] = df['revenue'].replace(0, np.nan)
df['revenue'] = df['revenue'].fillna(df['worldwide_revenue'])
df['revenue'] = df['revenue'].fillna(0).astype(np.int64)


df['budget'] = df['budget'].replace(0, np.nan)
df['budget'] = df['budget'].fillna(df['budget_extra'])
df['budget'] = df['budget'].fillna(0).astype(np.int64)


df = df.drop(columns=['worldwide_revenue', 'budget_extra'])


# Cleaning final data after merge

In [ ]:
df['release_date'] = df['release_date'].replace('Missing value',np.nan)
df['runtime'] = df['runtime'].replace(0,np.nan)
df['revenue'] = df['revenue'].replace(0,np.nan)
df['budget'] = df['budget'].replace(0,np.nan)
df['budget'] = df['budget'].astype('Int64')
df['revenue'] = df['revenue'].astype('Int64')
df['runtime'] = df['runtime'].astype('Int64')
df['release_date'] = pd.to_datetime(df['release_date'])

/tmp/ipython-input-3322449121.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['release_date'] = df['release_date'].replace('Missing value',np.nan)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191614 entries, 0 to 191613
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   imdb_id                  191614 non-null  object        
 1   tmdb_id                  191614 non-null  int64         
 2   title                    191614 non-null  object        
 3   release_date             188802 non-null  datetime64[ns]
 4   runtime                  175081 non-null  Int64         
 5   genres                   191614 non-null  object        
 6   director                 191614 non-null  object        
 7   top_cast                 191614 non-null  object        
 8   original_language        191614 non-null  object        
 9   production_companies     191614 non-null  object        
 10  budget                   23099 non-null   Int64         
 11  revenue                  44501 non-null   Int64         
 12  status          

In [ ]:
df.head()

,imdb_id,tmdb_id,title,release_date,runtime,genres,director,top_cast,original_language,production_companies,budget,revenue,status,actor_1_score,actor_2_score,actor_3_score,actor_4_score,actor_5_score,director_score,average_actor_score,top_actor_score,number_of_actors,main_production_country
0,tt0015724,286375,Dama de Noche,1993-05-18,96,"Drama, Mystery, Romance, Thriller",Eva López Sánchez,"Rafael Sánchez Navarro, Cecilia Toussaint, Mig...",es,Centro de Capacitación Cinematográfica (CCC),<NA>,<NA>,Released,13,7,12,16,70,2,23.6,70,5,Mexico
1,tt0035423,11232,Kate & Leopold,2001-12-25,118,"Romance, Comedy, Fantasy",James Mangold,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",en,"Konrad Pictures, Miramax",48000000,76019048,Released,51,43,58,31,31,13,42.8,58,5,United States of America
2,tt0036606,73069,"Another Time, Another Place",1983-05-13,96,"Drama, Romance, War",Michael Radford,"Phyllis Logan, Giovanni Mauriello, Denise Coff...",en,"Umbrella, Associated-Rediffusion Television, T...",<NA>,<NA>,Released,4,0,20,22,1,3,9.4,22,5,United Kingdom
3,tt0039442,133498,"Speak, Silent One",1973-06-01,88,Drama,Manuel Gutiérrez Aragón,"José Luis López Vázquez, Kiti Mánver, Hanna Ha...",es,"Filmverlag der Autoren, Elías Querejeta PC",<NA>,<NA>,Released,144,2,0,10,2,3,31.6,144,5,Spain
4,tt0044952,235186,The Citizen,1977-09-20,112,Drama,Ritwik Ghatak,"Satindra Bhattacharya, Prabhadevi, Sova Sen, K...",bn,Missing value,<NA>,<NA>,Released,10,4,26,12,1,48,10.6,26,5,India


# Data cleaning complete, exporting

In [ ]:
df.to_csv('cleaned_data.csv', index=False)